In [1]:

import IPython
import pandas as pd
import numpy as np

from datetime import datetime, date
from math import *
import json
%matplotlib inline
from decimal import *
getcontext().prec = 6
import re
from decimal import Decimal

from os import listdir
from os.path import isfile, join
from datetime import timedelta

In [2]:
# df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']


## Load historical Quote/Base prices

In [81]:

# df_btc = load_candles('BTC_GBP')
# df_eth = load_candles('ETH_GBP')
# df_usdt = load_candles('GBP_USDT')
# df_usdt = pd.DataFrame({'time': df_usdt.time, 'ohl3': df_usdt.ohl3**(-1)})

,time,avg
time,,
BNB_GBP-202009010100,2020-09-01 01:00:00,17.450000
BNB_GBP-202009010101,2020-09-01 01:01:00,17.450000
BNB_GBP-202009010102,2020-09-01 01:02:00,17.450000
BNB_GBP-202009010103,2020-09-01 01:03:00,17.450000
BNB_GBP-202009010104,2020-09-01 01:04:00,17.450000
...,...,...
BNB_GBP-202306170137,2023-06-17 01:37:00,185.800000
BNB_GBP-202306170138,2023-06-17 01:38:00,185.800000
BNB_GBP-202306170139,2023-06-17 01:39:00,185.900000


In [ ]:

df_trades['ntime'] = df_trades.mtime.dt.strftime('%Y%m%d%H%M').astype(int)

## Load Binance data

In [454]:
# Fee is not included in the total

df_binance = pd.concat([
    pd.read_csv(filepath, parse_dates=True ) for filepath in [
        './trading-data/binance/trading-history-2020.csv',
        './trading-data/binance/trading-history-2021.csv',
        './trading-data/binance/trading-history-2022.csv'
    ]
], axis=0)

df_binance.columns = ['date', 'pair', 'side', 'price', 'executed', 'amount', 'fee']
df_binance['time'] = pd.to_datetime(df_binance['date'], utc=True).dt.floor('S').dt.tz_convert('Europe/London')#.dt.floor('s')
df_binance.price = df_binance.price.str.replace(',','').astype('float')


re_pair = re.compile('(?P<base>[a-z]+)(?P<quote>GBP|USDT|ETH|BTC|USDC|DAI|BUSD)', re.IGNORECASE)
re_curr_amout = re.compile('^(?P<amt>[\d\.,]+)(?P<curr>\w+)$')

df_binance[['base', 'quote']] = df_binance.pair.str.extract(re_pair, expand=True)
df_binance[['executed', 'executed_curr']]  = df_binance.executed.str.split(re_curr_amout, expand=True).iloc[:,[1,2]]
df_binance[['amount', 'amount_curr']]  = df_binance.amount.str.split(re_curr_amout, expand=True).iloc[:,[1,2]]
df_binance[['fee', 'fee_curr']]  = df_binance.fee.str.split(re_curr_amout, expand=True).iloc[:,[1,2]]
for nc in ['executed', 'fee', 'amount']:
    df_binance[nc] = pd.to_numeric(df_binance[nc].str.replace(',', ''))
# df_binance['executed', 'fee', 'amount'] = 
# pd.to_numeric(df_binance[['executed', 'fee', 'amount']].values())
df_binance['exchange'] = 'binance'
# df_binance.tail()d
### !!! fee is not included in amount

# Checks
print('Checks', json.dumps({
    '# records': df_binance.shape[0],
    'base.isnull()' : df_binance[df_binance.base.isnull()].shape[0],
    'quote.isnull()' : df_binance[df_binance.quote.isnull()].shape[0],
    'price * executed != amount' : df_binance[(df_binance.price * df_binance.executed - df_binance.amount).abs() > 0.00001].shape[0],
    'executed_curr != base' : df_binance[df_binance.executed_curr != df_binance.base].shape[0],
    'amount_curr != quote' : df_binance[df_binance.amount_curr != df_binance.quote].shape[0],
    'fee_curr != quote & fee_curr != base' : df_binance[(df_binance.fee_curr != df_binance.quote) & (df_binance.fee_curr != df_binance.base)].shape[0],
    'price, amount, executed, fee < 0' : int((df_binance[['price', 'amount', 'executed', 'fee']] < 0).sum().sum())
}, indent=2))
df_binance.rename({'amount': 'total', 'executed': 'size'}, axis=1, inplace=True)
# df_binance.drop(['executed_curr', 'amount_curr', 'pair'], inplace=True, axis=1 )
df_binance = df_binance[['time', 'exchange', 'pair', 'base', 'quote', 'side', 'price', 'size', 'total', 'fee', 'fee_curr']]
df_binance

Checks {
  "# records": 26578,
  "base.isnull()": 0,
  "quote.isnull()": 0,
  "price * executed != amount": 0,
  "executed_curr != base": 0,
  "amount_curr != quote": 0,
  "fee_curr != quote & fee_curr != base": 5122,
  "price, amount, executed, fee < 0": 0
}


,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr
0,2020-12-30 02:01:29+00:00,binance,BTCGBP,BTC,GBP,BUY,20430.00,0.150000,3064.500000,0.0,BTC
1,2020-12-30 02:01:29+00:00,binance,BTCGBP,BTC,GBP,BUY,20430.00,0.000003,0.061290,0.0,BTC
2,2020-12-30 02:01:28+00:00,binance,BTCGBP,BTC,GBP,BUY,20430.00,0.100000,2043.000000,0.0,BTC
3,2020-12-30 02:01:28+00:00,binance,BTCGBP,BTC,GBP,BUY,20430.00,0.634890,12970.802700,0.0,BTC
4,2020-12-30 02:01:28+00:00,binance,BTCGBP,BTC,GBP,BUY,20430.00,0.150000,3064.500000,0.0,BTC
...,...,...,...,...,...,...,...,...,...,...,...
15,2022-09-16 23:13:15+01:00,binance,BTCGBP,BTC,GBP,BUY,17291.62,0.018270,315.917897,0.0,BNB
16,2022-09-11 17:20:07+01:00,binance,BTCUSDT,BTC,USDT,BUY,21694.29,0.025000,542.357250,0.0,BNB
17,2022-09-11 17:20:07+01:00,binance,BTCUSDT,BTC,USDT,BUY,21694.41,0.195990,4251.887416,0.0,BNB
18,2022-09-11 17:20:07+01:00,binance,BTCUSDT,BTC,USDT,BUY,21694.29,0.120000,2603.314800,0.0,BNB


In [58]:
# df_binance[(df_binance.fee_curr != df_binance.quote) & (df_binance.fee != 0) & (df_binance.fee_curr != df_binance.base)].fee_curr.unique()
df_binance[(df_binance.fee_curr == df_binance.quote)]
# df_binance[['fee', 'fee_curr']][df_binance.fee != 0].groupby('fee_curr').sum().round()
# df_binance['date'].dt.floor('S')
# df_binance.quote.unique()
# (df_binance[['price', 'size', 'total', 'fee']] < 0).sum().sum()

,time,ntime,exchange,pair,base,quote,side,price,size,total,fee,fee_curr
6,2020-12-30 00:57:56+00:00,202012300057,binance,BTCGBP,BTC,GBP,SELL,20450.0,0.025948,530.63660,0.0,GBP
7,2020-12-30 00:57:56+00:00,202012300057,binance,BTCGBP,BTC,GBP,SELL,20450.0,0.000002,0.04090,0.0,GBP
8,2020-12-30 00:57:56+00:00,202012300057,binance,BTCGBP,BTC,GBP,SELL,20450.0,0.109506,2239.39770,0.0,GBP
9,2020-12-30 00:57:56+00:00,202012300057,binance,BTCGBP,BTC,GBP,SELL,20450.0,0.001285,26.27825,0.0,GBP
10,2020-12-30 00:57:56+00:00,202012300057,binance,BTCGBP,BTC,GBP,SELL,20450.0,0.003635,74.33575,0.0,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...
26109,2021-01-04 21:25:41+00:00,202101042125,binance,BTCGBP,BTC,GBP,SELL,23170.0,0.000006,0.13902,0.0,GBP
26110,2021-01-04 21:25:35+00:00,202101042125,binance,BTCGBP,BTC,GBP,SELL,23170.0,0.861400,19958.63800,0.0,GBP
26111,2021-01-04 21:25:35+00:00,202101042125,binance,BTCGBP,BTC,GBP,SELL,23170.0,0.073700,1707.62900,0.0,GBP
26112,2021-01-04 21:25:35+00:00,202101042125,binance,BTCGBP,BTC,GBP,SELL,23170.0,0.150000,3475.50000,0.0,GBP


In [36]:
20228.62	* 0.113151

2288.88858162

In [26]:
dada = df_binance[(df_binance.fee != 0) & (df_binance.fee_curr == 'ADA')]
dada['ttot'] = dada.size * dada.price
dada

/tmp/ipykernel_4420/2318426777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dada['ttot'] = dada.size * dada.price


,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ttot
331,2020-12-26 01:31:12+00:00,binance,ADABTC,ADA,BTC,BUY,0.000006,41770.0,0.263151,41.7700,ADA,0.046777
332,2020-12-26 01:31:12+00:00,binance,ADABTC,ADA,BTC,BUY,0.000006,3933.0,0.024778,3.9330,ADA,0.046777
333,2020-12-26 01:31:12+00:00,binance,ADABTC,ADA,BTC,BUY,0.000006,2038.0,0.012839,2.0380,ADA,0.046777
53,2021-11-21 14:44:00+00:00,binance,ADAUSDT,ADA,USDT,BUY,1.869000,1530.5,2860.504500,1.5305,ADA,13877.325000
54,2021-11-21 14:44:00+00:00,binance,ADAUSDT,ADA,USDT,BUY,1.869000,160.5,299.974500,0.1605,ADA,13877.325000
...,...,...,...,...,...,...,...,...,...,...,...,...
20559,2021-02-06 18:21:12+00:00,binance,ADAUSDT,ADA,USDT,BUY,0.566000,2674.0,1513.484000,2.6740,ADA,4202.550000
20560,2021-02-06 18:21:03+00:00,binance,ADAUSDT,ADA,USDT,BUY,0.566000,25.3,14.319800,0.0253,ADA,4202.550000
20561,2021-02-06 18:21:03+00:00,binance,ADAUSDT,ADA,USDT,BUY,0.566000,444.5,251.587000,0.4445,ADA,4202.550000
20562,2021-02-06 18:21:03+00:00,binance,ADAUSDT,ADA,USDT,BUY,0.566000,25.8,14.602800,0.0258,ADA,4202.550000


In [27]:
0.566000 * 2674.0

1513.484

## Load Coinbase Pro data

In [41]:
df_coinbase = pd.read_csv('./trading-data/coinbase-pro/fills.csv', parse_dates=True) 
df_coinbase.rename({'product': 'pair', 
                    'created at': 'date',
                    }, inplace=True, axis='columns')
df_coinbase.pair = df_coinbase.pair.str.replace('-', '')
df_coinbase['time'] = pd.to_datetime(df_coinbase.date).dt.floor('S').dt.tz_convert(tz='Europe/London').dt.floor('s')
df_coinbase['size'] = df_coinbase['size'].astype('float') 
df_coinbase.total = df_coinbase.total.abs()
re_pair = re.compile('(?P<base>[a-z]+)(?P<quote>GBP|USDT|ETH|BTC|USDC|DAI|BUSD)', re.IGNORECASE)

df_coinbase[['base', 'quote']] = df_coinbase.pair.str.extract(re_pair, expand=True)

df_coinbase['exchange'] = 'coinbase-pro'
print({
    '# records': df_coinbase.shape[0],
    ### Total includes fee
    'price * size != total' : df_coinbase[(df_coinbase.price * df_coinbase.size - df_coinbase.total).abs() > 0.000001].shape[0],
    'price/fee/total unit != quote' : df_coinbase[df_coinbase['price/fee/total unit'] != df_coinbase.quote].shape[0],
    'size unit != base' : df_coinbase[df_coinbase['size unit'] != df_coinbase.base].shape[0],
    "'size', 'price', 'total', 'fee' < 0" : df_coinbase[['size', 'price', 'total', 'fee']].lt(0).sum().sum()

})
# base on the above observations, remove columns
df_coinbase.drop(['portfolio', 'trade id', 'price/fee/total unit', 'size unit'], inplace=True, axis=1)
df_coinbase['fee_curr'] = df_coinbase.quote
# df_coinbase['size'] = df_coinbase.size.abs()

df_coinbase = df_coinbase[['time', 'exchange', 'pair', 'base', 'quote', 'side', 'price', 'size', 'total', 'fee', 'fee_curr']]


# df_coinbase.shape, df_coinbase.dtypes,

df_coinbase


{'# records': 576, 'price * size != total': 576, 'price/fee/total unit != quote': 0, 'size unit != base': 0, "'size', 'price', 'total', 'fee' < 0": 0}


,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr
0,2021-09-24 00:12:51+01:00,coinbase-pro,DOTGBP,DOT,GBP,SELL,23.894,9.839000,233.917601,1.175465,GBP
1,2020-11-23 23:45:46+00:00,coinbase-pro,XRPGBP,XRP,GBP,BUY,0.455,10000.000000,4554.550000,4.550000,GBP
2,2020-11-24 15:10:43+00:00,coinbase-pro,XRPGBP,XRP,GBP,BUY,0.490,4959.000000,2432.339910,2.429910,GBP
3,2020-11-24 15:10:43+00:00,coinbase-pro,XRPGBP,XRP,GBP,BUY,0.490,5041.000000,2472.560090,2.470090,GBP
4,2020-11-24 15:48:12+00:00,coinbase-pro,XRPGBP,XRP,GBP,SELL,0.500,10010.000000,4999.995000,5.005000,GBP
...,...,...,...,...,...,...,...,...,...,...,...
571,2020-12-25 21:44:42+00:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,17920.000,0.943705,16924.714484,13.528948,GBP
572,2021-01-04 18:57:23+00:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,23000.000,0.845410,19463.883409,19.444439,GBP
573,2021-01-04 18:57:24+00:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,23000.000,0.240110,5528.060818,5.522538,GBP
574,2022-09-22 20:07:29+01:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.350,0.094000,1616.510113,9.641213,GBP


In [31]:
df_coinbase.fee_curr.unique()
df_coinbase[df_coinbase.fee_curr != df_coinbase.quote]

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr


In [29]:
23.894	* 9.839000, 233.917601	+ 1.175465
# df_coinbase['tot'] = df_coinbase['price'] * df_coinbase['size']
# df_coinbase.head()
# round(df_coinbase['price'] * df_coinbase['size'] - df_coinbase['total'] + df_coinbase['fee'], 4)
# df_coinbase['date'].dt.tz_convert('Europe/London')

(235.093066, 235.093066)

### Load Kucoin Data

In [8]:
# import warnings

# with warnings.catch_warnings(record=True):
# warnings.simplefilter("always")
    # myexcelfile = pd.read_excel(easy_payfile, engine="openpyxl")

df_kucoin = pd.concat([
    pd.read_excel('./trading-data/kucoin/BillingHistory-2021/Spot Orders_Completed Trades20230131.xlsx', parse_dates=True),
    # pd.read_csv('./trading-data/kucoin/BillingHistory-2022/Spot Orders_Completed Trades20230131.xlsx', parse_dates=True)
], axis=0)

df_kucoin.rename({'Symbol': 'pair', 
                    'Fee Currency': 'fee_curr',
                    'Avg. Filled Price' : 'price',
                    'Filled Amount': 'size',
                    'Filled Volume': 'total',
                    'Side': 'side',
                    'Fee': 'fee',
                    'Filled Time(UTC+08:00)': 'date',
                    }, inplace=True, axis='columns')
df_kucoin.pair = df_kucoin.pair.str.replace('-', '')
df_kucoin['time'] = pd.to_datetime(df_kucoin.date, infer_datetime_format=True, utc=True).dt.floor('S').dt.tz_convert(tz='Europe/London')
re_pair = re.compile('(?P<base>[a-z]+)(?P<quote>GBP|USDT|ETH|BTC|USDC|DAI|BUSD|UST)', re.IGNORECASE)

df_kucoin[['base', 'quote']] = df_kucoin.pair.str.extract(re_pair, expand=True)

df_kucoin['exchange'] = 'kucoin'
print({
    '# records': df_kucoin.shape[0],
    ### Total includes fee
    'size * price != total' : df_kucoin[(df_kucoin['price'] * df_kucoin['size'] - df_kucoin.total).abs() > 0.0001].astype(int).sum().sum(),
    'fee_curr != quote' : df_kucoin[df_kucoin['fee_curr'] != df_kucoin.quote].shape[0],
    # "'size', 'price', 'total', 'fee' < 0" : df_coinbase[['size', 'price', 'total', 'fee']].lt(0).sum().sum()

})
# base on the above observations, remove columns
# df_coinbase.drop(['portfolio', 'trade id', 'price/fee/total unit', 'size unit'], inplace=True, axis=1)
# df_coinbase['fee_curr'] = df_coinbase.quote
# df_coinbase['size'] = df_coinbase.size.abs()
df_kucoin['exchange'] = 'kucoin'
df_kucoin = df_kucoin[['time', 'exchange', 'pair', 'base', 'quote', 'side', 'price', 'size', 'total', 'fee', 'fee_curr']]

# df_kucoin.shape, 
df_kucoin.head()


{'# records': 55, 'size * price != total': 0.0, 'fee_curr != quote': 0}


/home/grenada/miniconda3/envs/py311/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/tmp/ipykernel_4420/2654441353.py:22: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_kucoin['time'] = pd.to_datetime(df_kucoin.date, infer_datetime_format=True, utc=True).dt.floor('S').dt.tz_convert(tz='Europe/London')


,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr
0,2021-08-19 22:21:31+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,24.1864,0.05,1.209320,0.001209,USDT
1,2021-08-20 01:02:28+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.7575,0.05,1.287875,0.001288,USDT
2,2021-08-20 01:19:51+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.3844,0.01,0.253844,0.000254,USDT
3,2021-08-20 01:28:19+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.4126,0.01,0.254126,0.000254,USDT
4,2021-08-20 01:28:25+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.4067,0.01,0.254067,0.000254,USDT


In [40]:
# df_kucoin[df_kucoin.fee_curr != df_kucoin.quote]
# df_kucoin['date'].dt.tz_convert(tz='Europe/London')

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr


## Consolidate trades across exchanges

In [455]:
df_trades = pd.concat([df_binance, df_coinbase, df_kucoin], axis=0)
df_trades.time = df_trades.time.dt.tz_localize(None)
# 
# Drop total as it will be calculated inc fees
# df_trades.drop(['total'], axis=1, inplace=True)
df_trades.sort_values(by='time', ascending=True, inplace=True)
df_trades.reset_index(inplace=True, drop=True)
df_trades['ctotal'] = df_trades['size'] * df_trades.price

# calculate fee in quote currency
# df_trades['fee_q'] = np.where(
#     df_trades.fee_curr == df_trades.quote, df_trades.fee, 
#     np.where(df_trades.fee_curr == df_trades.base, df_trades.fee * df_trades.price, pd.NA)
#     )

# df_trades['date'] = df_trades.time.dt.floor('d').dt.tz_localize(None)
# df_trades['ndate30p'] = (df_trades.date + timedelta(days=30)).dt.strftime('%Y%m%d').astype(int)
# df_trades['ndate30m'] = (df_trades.date - timedelta(days=30)).dt.strftime('%Y%m%d').astype(int)
# df_trades['ndate'] = df_trades.date.dt.strftime('%Y%m%d').astype(int)

# df_trades.set_index('time', inplace=True, drop=True)
# df_trades['tridx'] = df_trades.index
# df_trades['calc_total'] = df_trades['size'] * df_trades.price
# df_trades['mtime'] = df_trades.time.dt.tz_localize(None).dt.floor(freq='T')
df_trades['ntime'] = df_trades.time.dt.strftime('%Y%m%d%H%M').astype(int)
df_trades

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,ntime
0,2020-09-15 12:24:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8420.00,0.010000,84.621000,0.421000,GBP,84.200000,202009151224
1,2020-09-15 15:48:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8400.00,0.010000,84.420000,0.420000,GBP,84.000000,202009151548
2,2020-09-15 17:21:13,coinbase-pro,BTCGBP,BTC,GBP,BUY,8375.00,0.010000,84.168750,0.418750,GBP,83.750000,202009151721
3,2020-09-17 00:44:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8480.00,0.010000,85.224000,0.424000,GBP,84.800000,202009170044
4,2020-09-17 00:55:05,coinbase-pro,ETHGBP,ETH,GBP,BUY,282.00,0.500000,141.705000,0.705000,GBP,141.000000,202009170055
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27204,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17295.79,0.474600,8208.581934,0.000000,BNB,8208.581934,202209162320
27205,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17300.00,0.061490,1063.777000,0.000000,BNB,1063.777000,202209162320
27206,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17296.68,0.009330,161.378024,0.000000,BNB,161.378024,202209162320
27207,2022-09-22 20:07:29,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.35,0.094000,1616.510113,9.641213,GBP,1606.868900,202209222007


In [ ]:
df_trades[df_trades.base == 'GBP']

## Calculate prices and fees in GBP

In [232]:

trade_n_times = sorted(df_trades.ntime.unique())
# trade_n_times

### Quote currency prices in GBP

-load minute data for quote currencies from an aggregator

In [234]:
### Load 

# def load_candles(filename):

def load_quote_hist(filename=None, ohlcv=None, ntimes=None):
  df = None
  if ohlcv is not None:
    df = pd.DataFrame(ohlcv, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
  else:
    df = pd.read_json(filename)
    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
  # filename = f'./freq-user-data/data/{exchange.lower()}/{pair}-1m.json'
  df['time'] = pd.to_datetime(df['time'], unit='ms', utc=True).dt.tz_convert('Europe/London').dt.tz_localize(None)
  df['ntime'] = df.time.dt.strftime('%Y%m%d%H%M').astype(int)
  df.set_index('time', inplace=True, drop=True)
  
  df['ohl3'] = (df.open + df.high + df.low) / 3.
  df.drop(['open', 'high', 'low', 'close', 'volume'], axis=1, inplace=True)
  if ntimes is not None: df = df[df.ntime.isin(ntimes)]
  # df.drop(['timestamp', 'high', 'low', 'close', 'volume'], axis=1, inplace=True)
  # df['ohl3'] = (df['open'] + df['high'] + df['low']) / 3
  return df

qdf_eth_gbp  = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='ETH_GBP', exchange='binance'), ntimes=trade_n_times)
qdf_bnb_gbp  = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='BNB_GBP', exchange='binance'), ntimes=trade_n_times)
qdf_btc_gbp  = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='BTC_GBP', exchange='binance'), ntimes=trade_n_times)
qdf_gbp_usdt = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='GBP_USDT', exchange='binance'), ntimes=trade_n_times)

# qdf_gbp_usdt = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='GBP_USDT', exchange='binance'), ntimes=trade_n_times)
# qdf_gbp_usdt = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='GBP_USDT', exchange='binance'), ntimes=trade_n_times)
# qdf_gbp_usdt = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='GBP_USDT', exchange='binance'), ntimes=trade_n_times)
# qdf_gbp_usdt = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='GBP_USDT', exchange='binance'), ntimes=trade_n_times)
# 'UST', 'BUSD', 'DAI', 'USDC


In [334]:
df_q = pd.concat([
    df['ohl3'] for df in [qdf_btc_gbp, qdf_eth_gbp, qdf_bnb_gbp, qdf_gbp_usdt]
], axis=1, keys=['BTC', 'ETH', 'BNB', 'USDT'])

df_q['ntime'] = df_q.index.strftime('%Y%m%d%H%M').astype(int)
df_q['USDT'] = df_q.USDT ** -1

In [335]:
# Check for missing historical data
df_trades[~df_trades.ntime.isin(df_q.ntime)]

,time,exchange,pair,base,quote,side,price,size,fee,fee_curr,fee_q,ntime,gprice
556,2020-12-25 02:51:57,coinbase-pro,ETHGBP,ETH,GBP,SELL,450.00,3.120000,1.123200,GBP,1.1232,202012250251,450.00
557,2020-12-25 02:51:57,coinbase-pro,ETHGBP,ETH,GBP,SELL,450.00,0.663350,0.537313,GBP,0.537313,202012250251,450.00
558,2020-12-25 02:51:57,coinbase-pro,ETHGBP,ETH,GBP,SELL,450.18,7.015712,5.685000,GBP,5.685,202012250251,450.18
559,2020-12-25 02:51:57,coinbase-pro,ETHGBP,ETH,GBP,SELL,450.21,4.000000,3.241512,GBP,3.241512,202012250251,450.21
560,2020-12-25 02:52:16,coinbase-pro,ETHGBP,ETH,GBP,SELL,450.00,0.000938,0.000338,GBP,0.000338,202012250252,450.00


In [336]:
# Manually get the missing historical klines from another exchange
import ccxt
ex = ccxt.coinbasepro()

df_append = pd.concat(
    [
        load_quote_hist(ohlcv=ohlcv) for ohlcv in 
        [ ex.fetch_ohlcv('ETH/GBP', '1m', limit=1, since=ex.parse8601(str(t))) for t in np.unique(df_trades[~df_trades.ntime.isin(df_q.ntime)].time.values.astype('<M8[m]').astype('<M8[s]')) ]
    ]
).rename({'ohl3': 'ETH'}, axis=1, inplace=False)

df_q = pd.concat([df_q, df_append])
df_q.sort_index(inplace=True)
df_q = df_q[~df_q.index.duplicated(keep='first')]


In [337]:
df_append

,ntime,ETH
time,,
2020-12-25 02:51:00,202012250251,450.140000
2020-12-25 02:52:00,202012250252,450.233333


In [338]:
# Check again for missing historical data
np.unique(df_trades[~df_trades.ntime.isin(df_q.ntime)].time.values.astype('<M8[m]')).astype('<M8[s]')

array([], dtype='datetime64[s]')

### Calculate quote price and fee in GBP

In [339]:
df_q

,BTC,ETH,BNB,USDT,ntime
time,,,,,
2020-09-15 12:24:00,8445.000000,295.00,22.740,0.779120,202009151224
2020-09-15 15:48:00,8400.026667,286.19,22.100,0.782228,202009151548
2020-09-15 17:21:00,8387.576667,283.84,22.042,0.780945,202009151721
2020-09-17 00:44:00,8506.130000,283.88,21.659,0.776096,202009170044
2020-09-17 00:55:00,8496.550000,283.74,21.636,0.776096,202009170055
...,...,...,...,...,...
2022-09-11 17:20:00,18726.283333,1532.25,254.700,0.863061,202209111720
2022-09-16 23:13:00,17301.206667,1262.92,240.200,0.876168,202209162313
2022-09-16 23:14:00,17291.713333,1261.62,240.200,0.875912,202209162314


In [456]:
# calculate price and fee in GBP

for row in df_trades.itertuples():
  # base, quote, fee_curr, price, fee = row.base, row.quote, row.fee_curr, row.price, row.fee
  # print(f'Processing row: {row}')
  quote = row.quote
  if quote in ['UST', 'BUSD', 'DAI', 'USDC'] : quote = 'USDT'
  gprice = pd.NA

  if row.quote == 'GBP':
    gprice = row.price
  elif row.base == 'GBP':
    gprice = row.price ** -1
  else:
    gprice = row.price * df_q.loc[row.time.replace(second=0), quote]

  gfee = pd.NA
  if row.fee_curr == 'GBP': 
    gfee = row.fee
  elif row.fee_curr == row.quote:
    gfee = row.fee * df_q.loc[row.time.replace(second=0), quote]
  elif row.fee_curr == row.base:
    gfee = row.fee * gprice
  else:
    gfee = row.fee * df_q.loc[row.time.replace(second=0), row.fee_curr]

  df_trades.at[row.Index,'gfee'] = round(gfee,2)
  df_trades.at[row.Index,'gprice'] = gprice

df_trades

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,ntime,gfee,gprice
0,2020-09-15 12:24:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8420.00,0.010000,84.621000,0.421000,GBP,84.200000,202009151224,0.42,8420.00
1,2020-09-15 15:48:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8400.00,0.010000,84.420000,0.420000,GBP,84.000000,202009151548,0.42,8400.00
2,2020-09-15 17:21:13,coinbase-pro,BTCGBP,BTC,GBP,BUY,8375.00,0.010000,84.168750,0.418750,GBP,83.750000,202009151721,0.42,8375.00
3,2020-09-17 00:44:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8480.00,0.010000,85.224000,0.424000,GBP,84.800000,202009170044,0.42,8480.00
4,2020-09-17 00:55:05,coinbase-pro,ETHGBP,ETH,GBP,BUY,282.00,0.500000,141.705000,0.705000,GBP,141.000000,202009170055,0.70,282.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27204,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17295.79,0.474600,8208.581934,0.000000,BNB,8208.581934,202209162320,0.00,17295.79
27205,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17300.00,0.061490,1063.777000,0.000000,BNB,1063.777000,202209162320,0.00,17300.00
27206,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17296.68,0.009330,161.378024,0.000000,BNB,161.378024,202209162320,0.00,17296.68
27207,2022-09-22 20:07:29,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.35,0.094000,1616.510113,9.641213,GBP,1606.868900,202209222007,9.64,17094.35


In [410]:
# Check for unpopulated gfee and gprice (fee and price in GBP)
df_trades[df_trades.gprice.isna() | df_trades.gfee.isna()]

,time,exchange,pair,base,quote,side,price,size,fee,fee_curr,ntime,gfee,gprice


In [459]:
# calculate amount and total in GBP
df_trades['gamount'] = (df_trades['size'] * df_trades.gprice).round(2)
df_trades['gtotal'] = df_trades['gamount'] + df_trades['gfee'] * np.where(df_trades.side == 'BUY', 1, -1) * np.where(df_trades.base == 'GBP', -1, 1)
df_trades['gfprice'] = df_trades['gtotal'] / df_trades['size']
# df_trades[(df_trades.fee_curr == df_trades.quote) ].fee_curr.unique()
# df_trades.fee_curr.unique()
df_trades

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,ntime,gfee,gprice,gamount,gtotal,gfprice
0,2020-09-15 12:24:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8420.00,0.010000,84.621000,0.421000,GBP,84.200000,202009151224,0.42,8420.00,84.20,84.62,8462.000000
1,2020-09-15 15:48:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8400.00,0.010000,84.420000,0.420000,GBP,84.000000,202009151548,0.42,8400.00,84.00,84.42,8442.000000
2,2020-09-15 17:21:13,coinbase-pro,BTCGBP,BTC,GBP,BUY,8375.00,0.010000,84.168750,0.418750,GBP,83.750000,202009151721,0.42,8375.00,83.75,84.17,8417.000000
3,2020-09-17 00:44:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8480.00,0.010000,85.224000,0.424000,GBP,84.800000,202009170044,0.42,8480.00,84.80,85.22,8522.000000
4,2020-09-17 00:55:05,coinbase-pro,ETHGBP,ETH,GBP,BUY,282.00,0.500000,141.705000,0.705000,GBP,141.000000,202009170055,0.70,282.00,141.00,141.70,283.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27204,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17295.79,0.474600,8208.581934,0.000000,BNB,8208.581934,202209162320,0.00,17295.79,8208.58,8208.58,17295.785925
27205,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17300.00,0.061490,1063.777000,0.000000,BNB,1063.777000,202209162320,0.00,17300.00,1063.78,1063.78,17300.048788
27206,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17296.68,0.009330,161.378024,0.000000,BNB,161.378024,202209162320,0.00,17296.68,161.38,161.38,17296.891747
27207,2022-09-22 20:07:29,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.35,0.094000,1616.510113,9.641213,GBP,1606.868900,202209222007,9.64,17094.35,1606.87,1616.51,17196.914894


In [414]:
# check different cases are sane
df_trades[df_trades.fee_curr == df_trades.base]
df_trades[df_trades.fee_curr == df_trades.quote]
df_trades[(df_trades.base == 'GBP') & (df_trades.gfee != 0)]

,time,exchange,pair,base,quote,side,price,size,fee,fee_curr,ntime,gfee,gprice,gamount,gtotal,gfprice
2396,2021-01-16 19:43:43,binance,GBPUSDT,GBP,USDT,SELL,1.3551,140.0,0.003334,BNB,202101161943,0.1,0.737953,103.31,103.41,0.738643
2397,2021-01-16 19:43:43,binance,GBPUSDT,GBP,USDT,SELL,1.3552,140.0,0.003334,BNB,202101161943,0.1,0.737898,103.31,103.41,0.738643


## Create Acc / Disposal dataframe

In [475]:
acc_cols = ['time', 'ntime', 'base', 'quote', 'size', 'gfprice', 'gtotal']
acc_cross_cols = ['time', 'ntime', 'quote', 'total', 'gamount']
df_acc_dis = pd.concat([
df_trades[acc_cols][(df_trades.quote == 'GBP') & (df_trades.side == 'BUY')].assign(tpy='ACC').rename({'base': 'asset'}, axis=1).drop(['quote'], axis=1),
df_trades[acc_cols][(df_trades.quote == 'GBP') & (df_trades.side == 'SELL')].assign(tpy='DIS').rename({'base': 'asset'}, axis=1).drop(['quote'], axis=1),
df_trades[acc_cols][(df_trades.base == 'GBP') & (df_trades.side == 'BUY')].assign(tpy='DIS').rename({'quote': 'asset'}, axis=1).drop(['base'], axis=1),
df_trades[acc_cols][(df_trades.base == 'GBP') & (df_trades.side == 'SELL')].assign(tpy='ACC').rename({'quote': 'asset'}, axis=1).drop(['base'], axis=1),
df_trades[acc_cols][(df_trades.base != 'GBP') & (df_trades.quote != 'GBP') & (df_trades.side == 'BUY')].assign(tpy='ACC').rename({'base': 'asset'}, axis=1).drop(['quote'], axis=1),
df_trades[acc_cols][(df_trades.base != 'GBP') & (df_trades.quote != 'GBP') & (df_trades.side == 'SELL')].assign(tpy='DIS').rename({'base': 'asset'}, axis=1).drop(['quote'], axis=1),
df_trades[(df_trades.base != 'GBP') & (df_trades.quote != 'GBP') & (df_trades.side == 'BUY')][acc_cross_cols].assign(tpy='DIS').rename({'quote': 'asset', 'total': 'size', 'gamount': 'gtotal'}, axis=1).assign(gfprice=lambda x: x['gtotal'] / x['size']),
df_trades[(df_trades.base != 'GBP') & (df_trades.quote != 'GBP') & (df_trades.side == 'SELL')][acc_cross_cols].assign(tpy='ACC').rename({'quote': 'asset', 'total': 'size', 'gamount': 'gtotal'}, axis=1).assign(gfprice=lambda x: x['gtotal'] / x['size'])
],axis=0, ignore_index=True)


df_acc_dis['date'] = df_acc_dis.time.dt.floor('d').dt.tz_localize(None)
df_acc_dis['ndate30p'] = (df_acc_dis.date + timedelta(days=30)).dt.strftime('%Y%m%d').astype(int)
df_acc_dis['ndate30m'] = (df_acc_dis.date - timedelta(days=30)).dt.strftime('%Y%m%d').astype(int)
df_acc_dis

,time,ntime,asset,size,gfprice,gtotal,tpy,date,ndate30p,ndate30m
0,2020-09-15 12:24:15,202009151224,BTC,0.01,8462.000000,84.62,ACC,2020-09-15,20201015,20200816
1,2020-09-15 15:48:15,202009151548,BTC,0.01,8442.000000,84.42,ACC,2020-09-15,20201015,20200816
2,2020-09-15 17:21:13,202009151721,BTC,0.01,8417.000000,84.17,ACC,2020-09-15,20201015,20200816
3,2020-09-17 00:44:00,202009170044,BTC,0.01,8522.000000,85.22,ACC,2020-09-17,20201017,20200818
4,2020-09-17 00:55:05,202009170055,ETH,0.50,283.400000,141.70,ACC,2020-09-17,20201017,20200818
...,...,...,...,...,...,...,...,...,...,...
51295,2021-12-03 20:53:39,202112032053,USDT,4258.02,0.756239,3220.08,ACC,2021-12-03,20220102,20211103
51296,2021-12-03 20:53:39,202112032053,USDT,4140.36,0.756239,3131.10,ACC,2021-12-03,20220102,20211103
51297,2021-12-03 20:53:39,202112032053,USDT,2438.00,0.756239,1843.71,ACC,2021-12-03,20220102,20211103
51298,2021-12-03 20:53:39,202112032053,USDT,2041.56,0.756240,1543.91,ACC,2021-12-03,20220102,20211103


In [471]:
df_trades[(df_trades.base != 'GBP') & (df_trades.quote != 'GBP') & (df_trades.side == 'BUY')][acc_cross_cols].assign(tpy='DIS').rename({'quote': 'asset', 'total': 'size', 'gamount': 'gtotal'}, axis=1).assign(gfprice=lambda x: x['gtotal'] / x['size'])
df_trades[(df_trades.base != 'GBP') & (df_trades.quote != 'GBP') & (df_trades.side == 'SELL')][acc_cross_cols].assign(tpy='ACC').rename({'quote': 'asset', 'total': 'size', 'gamount': 'gtotal'}, axis=1).assign(gfprice=lambda x: x['gtotal'] / x['size'])

,time,ntime,asset,size,gtotal,tpy,gfprice
195,2020-11-26 02:00:54,202011260200,BTC,0.000505,7.13,ACC,14118.965652
196,2020-11-26 02:00:58,202011260200,BTC,0.332791,4700.04,ACC,14123.082345
197,2020-11-26 02:01:54,202011260201,BTC,0.000304,4.29,ACC,14116.680783
360,2020-12-09 00:05:11,202012090005,BTC,0.051067,701.24,ACC,13731.795883
361,2020-12-09 00:05:12,202012090005,BTC,0.000579,7.95,ACC,13732.455493
...,...,...,...,...,...,...,...
27166,2021-12-03 20:53:39,202112032053,USDT,4258.020000,3220.08,ACC,0.756239
27167,2021-12-03 20:53:39,202112032053,USDT,4140.360000,3131.10,ACC,0.756239
27168,2021-12-03 20:53:39,202112032053,USDT,2438.000000,1843.71,ACC,0.756239
27169,2021-12-03 20:53:39,202112032053,USDT,2041.560000,1543.91,ACC,0.756240


In [541]:
# pd.concat([
#     df_trades[(df_trades['base'] == 'GBP') | (df_trades['quote'] == 'GBP')],

# ])

### GBP - based buys and sells
df_quote_gbp_buy = df_trades[(df_trades['side'] == 'BUY') & (df_trades['quote'] == 'GBP')].copy()
df_quote_gbp_buy[['csize', 'cprice']] = df_quote_gbp_buy[['size', 'price']]
df_quote_gbp_buy['trtype'] = 'ACC'
df_quote_gbp_buy['cbase'] = df_quote_gbp_buy['base']
df_quote_gbp_buy['cquote'] = 'GBP'
df_quote_gbp_buy['qprice'] = 1.0

df_quote_gbp_sell = df_trades[(df_trades['side'] == 'BUY') & (df_trades['quote'] == 'GBP')].copy()
df_quote_gbp_sell[['csize', 'cprice', 'ctotal']] = df_quote_gbp_sell[['size', 'price', 'calc_total']]
df_quote_gbp_sell['trtype'] = 'DIS'
df_quote_gbp_sell['cbase'] = df_quote_gbp_sell['base']
df_quote_gbp_sell['cquote'] = 'GBP'
df_quote_gbp_sell['qprice'] = 1.0


### Reverse GBP buys and sells
# df_base_gbp_buy = df_trades[(df_trades['side'] == 'BUY') & (df_trades['base'] == 'GBP')]
# df_base_gbp_buy

df_base_gbp_sell = df_trades[(df_trades['side'] == 'SELL') & (df_trades['base'] == 'GBP')].copy()

# df_base_gbp_sell['csize', 'ctotal']] = df_base_gbp_sell[['total', 'size']]
df_base_gbp_sell['cprice'] = df_base_gbp_sell.price ** (-1)
df_base_gbp_sell[['csize']] = df_base_gbp_sell[['calc_total']]
df_base_gbp_sell['trtype'] = 'ACC'
df_base_gbp_sell['cbase'] = df_base_gbp_sell['quote']
df_base_gbp_sell['cquote'] = 'GBP'
df_base_gbp_sell['qprice'] = 1.0

# df_base_gbp_sell[['ctotal', 'csize']] = df_base_gbp_sell[['csize', 'ctotal']]

### Cross-coin buys
df_xxx_buy = df_trades[(df_trades['side'] == 'BUY') & (df_trades['base'] != 'GBP') & (df_trades['quote'] != 'GBP')]
df_xxx_buy_acc = df_xxx_buy.copy()
df_xxx_buy_acc['cbase'] = df_xxx_buy_acc['base']
df_xxx_buy_acc['cquote'] = df_xxx_buy_acc['quote']
df_xxx_buy_acc['trtype'] = 'ACC'
df_xxx_buy_acc['cprice'] = df_xxx_buy_acc.price 
df_xxx_buy_acc[['csize']] = df_xxx_buy_acc[['size']]

df_xxx_buy_dis = df_xxx_buy.copy()
df_xxx_buy_dis['cbase'] = df_xxx_buy_dis['quote']
# df_xxx_buy_dis['cquote'] = df_xxx_buy_dis['quote']
df_xxx_buy_dis['trtype'] = 'DIS'
# df_xxx_buy_dis['cprice'] = df_xxx_buy_dis.price ** (-1)
df_xxx_buy_dis[['csize']] = df_xxx_buy_acc[['calc_total']]



### Cross-coin sells
df_xxx_sell = df_trades[(df_trades['side'] == 'SELL') & (df_trades['base'] != 'GBP') & (df_trades['quote'] != 'GBP')]
df_xxx_sell_acc = df_xxx_sell.copy()
df_xxx_sell_acc['cbase'] = df_xxx_sell_acc['quote']
# df_xxx_sell_acc['cquote'] = df_xxx_buy_acc['quote']
df_xxx_sell_acc['trtype'] = 'ACC'
# df_xxx_sell_acc['cprice'] = df_xxx_sell_acc.price 
df_xxx_sell_acc[['csize']] = df_xxx_sell_acc[['calc_total']]

df_xxx_sell_dis = df_xxx_sell.copy()
df_xxx_sell_dis['cbase'] = df_xxx_sell_dis['base']
# df_xxx_sell_dis['cquote'] = df_xxx_sell_dis['quote']
df_xxx_sell_dis['trtype'] = 'DIS'
# df_xxx_sell_dis['cprice'] = df_xxx_sell_dis.price ** (-1)
df_xxx_sell_dis[['csize']] = df_xxx_sell_dis[['size']]


# df_xxx_buy.quote.unique()
# df_base_gbp_sell
df_tran = pd.concat([
    df_quote_gbp_buy, df_quote_gbp_sell, df_base_gbp_sell,
    df_xxx_buy_acc, df_xxx_buy_dis,
    df_xxx_sell_acc, df_xxx_sell_dis
], axis=0)

df_tran['cquote'] = np.where((
        (df_tran.cquote.isnull()) & 
        (df_tran.cbase.isin(['BTC', 'ETH', 'USDT']))
    ), 'GBP', df_tran.cquote 
)

fl_usdx = df_tran.cbase.isin(['DAI', 'USDT', 'USDC', 'UST', 'BUSD']) 

df_tran['cprice'] = np.where(
    (fl_usdx & df_tran.cprice.isnull()), 
    1., df_tran.cprice)


df_tran['cquote'] = np.where(
    fl_usdx & df_tran.cquote.isnull(), 
    'USDT', df_tran.cquote)

df_tran.to_csv('test.csv')


In [615]:
fl_buy = df_trades['side'] == 'BUY'
fl_sell = df_trades['side'] == 'SELL'
fl_base_gbp = df_trades['base'] == 'GBP'
fl_quote_gbp = df_trades['quote'] == 'GBP'
fl_x2x = (~fl_base_gbp & ~fl_quote_gbp)

df_tran = pd.concat([
    df_trades[fl_buy & fl_quote_gbp].assign(
        asset=lambda df: df['base'], op='ACC', 
        aquote='GBP',
        asize=lambda df: df['size'],
        aprice=lambda df: df['price'], 
        qprice=1.
    ),
    df_trades[fl_sell & fl_quote_gbp].assign(
        asset=lambda df: df['base'], op='DIS', 
        aquote='GBP',
        asize=lambda df: df['size'], 
        aprice=lambda df: df['price'], 
        qprice=1.
    ),

    df_trades[fl_buy & fl_base_gbp].assign(
        asset=lambda df: df['quote'], op='DIS', 
        aquote='GBP',
        asize=lambda df: df['calc_total'], 
        aprice=lambda df: df['price'] ** (-1), 
        qprice=1.
    ),
    df_trades[fl_sell & fl_base_gbp].assign(
        asset=lambda df: df['quote'], op='ACC', 
        aquote='GBP',
        asize=lambda df: df['calc_total'], 
        aprice=lambda df: df['price'] * (-1), 
        qprice=1.
    ),

    df_trades[fl_x2x & fl_buy].assign(
        asset=lambda df: df['base'], op='ACC', 
        aquote=lambda df: df['quote'],
        asize=lambda df: df['size'], 
        aprice=lambda df: df['price'], 
        # qprice=1.
    ),
    df_trades[fl_x2x & fl_buy].assign(
        asset=lambda df: df['quote'], op='DIS', 
        aquote=lambda df: df['quote'],
        asize=lambda df: df['calc_total'], 
        aprice=1., 
        # qprice=1.
    ),


    df_trades[fl_x2x & fl_sell].assign(
        asset=lambda df: df['quote'], op='ACC', 
        aquote=lambda df: df['quote'],
        asize=lambda df: df['calc_total'], 
        aprice=1, 
        # qprice=1.
    ),
    df_trades[fl_x2x & fl_sell].assign(
        asset=lambda df: df['base'], op='DIS', 
        aquote=lambda df: df['quote'],
        asize=lambda df: df['size'], 
        aprice=lambda df: df['price'], 
        # qprice=1.
    ),

], axis=0)

### Convert all stablecoins to USDT for simplicity
df_tran['aquote'] = np.where(df_tran.aquote.isin(['DAI', 'UST', 'BUSD', 'USDC']), 'USDT', df_tran.aquote)
# flt_x2x
df_tran['qsize'] = np.where(
    df_tran.quote == 'GBP',
    df_tran['calc_total'],
    np.where(
        df_tran.base == 'GBP',
        df_tran['size'],
        np.where(
            (
                ((df_tran['side'] == 'BUY') & (df_tran['op'] == 'ACC')) |
                ((df_tran['side'] == 'SELL') & (df_tran['op'] == 'DIS'))
            ),
            df_tran['calc_total'],
            # nan
            np.where(
                (
                    ((df_tran['side'] == 'BUY') & (df_tran['op'] == 'DIS')) |
                    ((df_tran['side'] == 'SELL') & (df_tran['op'] == 'ACC'))
                ),
                df_tran.calc_total,
                nan
            )
        )
    )    
)
# df_tran[df_tran['qsize'].isnull()].head()
df_tran.head()
# df_tran['mtime'] = df_tran.index.ceil('m')
# df_tran.to_csv('./test.csv')
# df_tran.tail()


,time,exchange,pair,base,quote,side,price,size,total,fee,...,tridx,calc_total,mtime,asset,op,aquote,asize,aprice,qprice,qsize
0,2020-09-15 12:24:15+01:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8420.0,0.01,84.62100,0.42100,...,0,84.20,2020-09-15 12:24:00,BTC,ACC,GBP,0.01,8420.0,1.0,84.20
1,2020-09-15 15:48:15+01:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8400.0,0.01,84.42000,0.42000,...,1,84.00,2020-09-15 15:48:00,BTC,ACC,GBP,0.01,8400.0,1.0,84.00
2,2020-09-15 17:21:13+01:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8375.0,0.01,84.16875,0.41875,...,2,83.75,2020-09-15 17:21:00,BTC,ACC,GBP,0.01,8375.0,1.0,83.75
3,2020-09-17 00:44:00+01:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8480.0,0.01,85.22400,0.42400,...,3,84.80,2020-09-17 00:44:00,BTC,ACC,GBP,0.01,8480.0,1.0,84.80
4,2020-09-17 00:55:05+01:00,coinbase-pro,ETHGBP,ETH,GBP,BUY,282.0,0.50,141.70500,0.70500,...,4,141.00,2020-09-17 00:55:00,ETH,ACC,GBP,0.50,282.0,1.0,141.00


In [17]:
def load_candles(pair):
  filename = f'./freq-user-data/data/binance/{pair}-1m.json'
  df = pd.read_json(filename)
  df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
  # df.columns = ['timestamp', 'open']
  df['time'] = pd.to_datetime(df['time'], unit='ms', utc=True).dt.tz_convert('Europe/London').dt.tz_localize(None)
  # df = df[(df['date']>=date(2020,4,6)) & (df['date']<=date(2021,4,5))] 
  # df['timestamp'] = df.timestamp.astype('datetime64[m]')
  # df.set_index(pd.DatetimeIndex(df["time"]), inplace=True, drop=True)
  # df = df['2020-04-06' : '2021-04-05']
  df['ohl3'] = (df.open + df.high + df.low) / 3.
  # df.drop(['timestamp', 'high', 'low', 'close', 'volume'], axis=1, inplace=True)
  # df['ohl3'] = (df['open'] + df['high'] + df['low']) / 3
  return df

df_btc = load_candles('BTC_GBP')
df_eth = load_candles('ETH_GBP')
df_usdt = load_candles('GBP_USDT')
df_usdt = pd.DataFrame({'time': df_usdt.time, 'ohl3': df_usdt.ohl3**(-1)})

# kline_pairs = [f.split('.')[0].split('-')[0] for f in listdir(datapath) if isfile(join(datapath, f))]
# klines = {pair: load_candles(f'./data/{pair}-1m.json') for pair in kline_pairs}
# klines


In [18]:
df_usdt

,time,ohl3
0,2020-09-01 01:00:00,0.752672
1,2020-09-01 01:01:00,0.752672
2,2020-09-01 01:02:00,0.752785
3,2020-09-01 01:03:00,0.752785
4,2020-09-01 01:04:00,0.752785
...,...,...
1297622,2023-02-20 01:47:00,0.831947
1297623,2023-02-20 01:48:00,0.831947
1297624,2023-02-20 01:49:00,0.831947
1297625,2023-02-20 01:50:00,0.832178


In [617]:

df_quotes = pd.concat([
    df_btc[['time', 'ohl3']].assign(asset='BTC'),
    df_eth[['time', 'ohl3']].assign(asset='ETH'),
    df_usdt[['time', 'ohl3']].assign(asset='USDT')
], axis=0)
df_quotes.rename(columns={'time': 'mtime', 'ohl3': 'qprice', 'asset': 'aquote'}, inplace=True)
df_quotes.set_index(['mtime', 'aquote'], inplace=True, drop=True)
df_quotes.sort_index(inplace=True)
# df_quotes[df_quotes]
df_quotes = df_quotes[~df_quotes.index.duplicated(keep='first')]
df_quotes.tail()

qprice
mtime               aquote              
2023-02-20 01:50:00 ETH      1390.953333
                    USDT        0.832178
2023-02-20 01:51:00 BTC     20199.150000
                    ETH      1392.000000
                    USDT        0.831947

In [627]:
# df_tran.set_index(['mtime', 'aquote'], inplace=True, )
print(df_tran.shape)
qdf_tran = df_tran.merge(df_quotes, how='left', on=['mtime', 'aquote'])
qdf_tran['qprice'] = np.where(qdf_tran.qprice_x.isnull(), qdf_tran.qprice_y, qdf_tran.qprice_x)
qdf_tran.to_csv('./test.csv')
qdf_tran[qdf_tran.qprice.isnull()]

(51300, 25)


,time,exchange,pair,base,quote,side,price,size,total,fee,...,mtime,asset,op,aquote,asize,aprice,qprice_x,qsize,qprice_y,qprice


In [589]:
fl_qprice_na = df_tran.qprice.isnull()
df_tran_qnull = df_tran[fl_qprice_na]

df_aquotes_ = pd.merge(left=df_tran_qnull[['mtime', 'aquote']], right=df_quotes, how='left', left_on=['mtime', 'aquote'], right_on=['time','asset'] )
# df_tran_qnull['qprice'] = np.where(df_tran_qnull.qprice.isnull(),
#                              df_aquotes_['ohl3'], df_tran_qnull.qprice)
df_tran_qnull['qprice'] = df_aquotes_['ohl3']


df_tran_q = pd.concat([
    df_tran_qnull,
    df_tran[~fl_qprice_na]
], axis=0)

# df_tran.shape, df_tran_q.shape
# df_tran_qnull.shape, df_aquotes_.shape
# df_tran = df_tran[~df_tran.qprice.isnull()]
# df_tran_qnull[df_tran_qnull.xprice.isnull()]
df_tran_q.to_csv('./test.csv')
df_tran_q[df_tran_q.qprice.isnull()]

/tmp/ipykernel_222316/1639742356.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tran_qnull['qprice'] = df_aquotes_['ohl3']


,time,exchange,pair,base,quote,side,price,size,total,fee,...,tridx,calc_total,mtime,asset,op,aquote,asize,aprice,qprice,qsize


In [592]:
df_btc[(abs(df_btc.ohl3 - 0.738188976377953)) < 0.00001]

,time,open,high,low,close,volume,ohl3


In [ ]:
df_tran.tail()

In [350]:
# lots = { "GBP": {"amt": 1000000, "total": 250000}}
lots = {}
total_pnl = 0.0
for row in df.itertuples():
  base, quote, amt, quote_total = row.base, row.quote, row.amt, row.total

  # if row.exchange != 'coinbase-pro':
  #   continue
  
  if not quote in lots:
    lots[quote] = {"amt": 0.0, "total": 0.0}
  if not base in lots:
    lots[base] = {"amt": 0.0, "total": 0.0}

  quote_price = 1
  if quote == "GBP":
    quote_price = 1
  elif quote == "BTC":
    quote_price = klines['BTC_GBP'].loc[row.minute]
  elif quote == "ETH":
    quote_price = klines['ETH_GBP'].loc[row.minute]
  elif  quote == "USDT":
    quote_price = 1. / klines['GBP_USDT'].loc[row.minute]

  trade_total = quote_total * quote_price

  df.at[row.Index,'q_price'] = quote_price
  df.at[row.Index,'t_total'] = trade_total

  base_pnl, quote_pnl = 0.0, 0.0

  lot_base_amt, lot_base_total = lots[base]['amt'], lots[base]['total']
  lot_quote_amt, lot_quote_total = lots[quote]['amt'], lots[quote]['total']
  lot_base_price = 1 if base == "GBP" else lot_base_total/lot_base_amt if lot_base_amt > 0.0 else 0.0
  lot_quote_price = 1 if quote == "GBP" else lot_quote_total/lot_quote_amt if lot_quote_amt > 0.0 else 0.0

  trade_base_cost = lot_base_price * amt
  trade_quote_cost = lot_quote_price * quote_total

  df.at[row.Index,'lba'] = lot_base_amt
  df.at[row.Index,'lbt'] = lot_base_total  
  df.at[row.Index,'lqa'] = lot_quote_amt
  df.at[row.Index,'lqt'] = lot_quote_total
  df.at[row.Index,'lbp'] = lot_base_price
  df.at[row.Index,'tbc'] = trade_base_cost
  df.at[row.Index,'tqc'] = trade_quote_cost

  quote_pnl, base_pnl, trade_pnl = 0.0, 0.0, 0.0

  if row.side == 'BUY':
    lots[base]['amt'] += amt
    lots[base]['total'] += trade_total   
    lots[quote]['amt'] -= quote_total 
    lots[quote]['total'] -= trade_quote_cost

    if quote != 'GBP': # and quote != 'USDT' :
        quote_pnl = (trade_total - trade_quote_cost)
  else: #sell
    lots[base]['amt'] -= amt
    lots[base]['total'] -= trade_base_cost   
    lots[quote]['amt'] += quote_total
    lots[quote]['total'] += trade_total

    if base != 'GBP': # and base != 'USDT' :
        base_pnl = (trade_total - trade_base_cost)

  trade_pnl = (base_pnl + quote_pnl)
  total_pnl += trade_pnl
  
  df.at[row.Index,'bpnl'] = base_pnl
  df.at[row.Index,'qpnl'] = quote_pnl
  df.at[row.Index,'trpnl'] = trade_pnl
  df.at[row.Index,'topnl'] = total_pnl

df.to_csv('all-trades.csv')
df[(df['base'] == 'BTC' ) | (df['quote'] == 'BTC') ].to_csv('btc-trades.csv')

df[(df['base'] == 'USDT' ) | (df['quote'] == 'USDT') ].to_csv('usdt-trades.csv')
int(total_pnl)

397599

In [495]:
# lots = { "GBP": {"amt": 1000000, "total": 250000}}
lots = {}
total_pnl = 0.0
for row in df.itertuples():
  base, quote, amt, q_total = row.base, row.quote, row.amt, row.q_total

  total = 0
  if quote == "GBP":
    total = q_total
  if base == "GBP":
    total = amt
  elif quote == "BTC":
    total = q_total * klines['BTC_GBP'].loc[row.minute]
  elif quote == "ETH":
    total = q_total * klines['ETH_GBP'].loc[row.minute]
  elif  quote == "USDT":
    total = q_total / klines['GBP_USDT'].loc[row.minute]

  # trade_total = quote_total * quote_price

  df.at[row.Index,'total'] = total

# df['price'] = df['']

df.to_csv('all-trades.csv')
# df[(df['base'] == 'BTC' ) | (df['quote'] == 'BTC') ].to_csv('btc-trades.csv')

# df[(df['base'] == 'USDT' ) | (df['quote'] == 'USDT') ].to_csv('usdt-trades.csv')
# int(total_pnl)

In [496]:
dfl = pd.concat(
  [df[(df['side'] == 'BUY')].assign(typ='acq', asset=df.base, qty=df.amt),
  df[(df['side'] == 'BUY')].assign(typ='dis', asset=df.quote, qty=df.q_total),
  df[(df['side'] == 'SELL')].assign(typ='acq', asset=df.quote, qty=df.q_total),
  df[(df['side'] == 'SELL')].assign(typ='dis', asset=df.base, qty=df.amt)]
).sort_values(by="timestamp", ascending=True, ignore_index=True)

dfl['price'] = dfl.total / dfl.qty
# dfl = dfl[dfl['exchange'] == 'coinbase-pro']
dfl.to_csv('acq-dis.csv')



In [497]:
lots = {}
tpnl = 0.0
for row in dfl.itertuples():
  if row.asset == 'GBP':
    continue
  if not row.asset in lots:
    lots[row.asset] = {"qty": 0.0, "total": 0.0}

  if row.typ == 'acq':
    lots[row.asset]['qty'] += row.qty
    lots[row.asset]['total'] += row.total

    dfl.at[row.Index,'Lqty'] = lots[row.asset]['qty']
    dfl.at[row.Index,'Ltotal'] = lots[row.asset]['total']
    dfl.at[row.Index,'Lprice'] = lots[row.asset]['total'] / lots[row.asset]['qty']

  elif row.typ == 'dis':
    cost_price = lots[row.asset]['total'] / lots[row.asset]['qty'] 
    cost = row.qty * cost_price
    lots[row.asset]['qty'] -= row.qty
    lots[row.asset]['total'] -= cost
    pnl = row.total - cost

    dfl.at[row.Index,'cost'] = cost
    dfl.at[row.Index,'Lqty'] = lots[row.asset]['qty']
    dfl.at[row.Index,'Ltotal'] = lots[row.asset]['total']
    dfl.at[row.Index,'Lprice'] = (lots[row.asset]['total'] / lots[row.asset]['qty'] if lots[row.asset]['qty'] > 0 else 0)
    if row.asset not in ['GBP']:
      dfl.at[row.Index,'pnl'] = pnl
      tpnl += pnl

tpnl


397598.58947253367

In [498]:
filter = (dfl['asset'] != 'GBP') & (dfl['typ'] == 'dis') # & (dfl['asset'] != 'USDT') #& (dfl['exchange'] == 'coinbase-pro')
dfr = dfl[filter].copy()
dfr['date'] = dfr.timestamp.dt.date
dfr.to_csv('pnl.csv')
len(dfr)

647

647

In [503]:
dfpt = dfr.copy()
dfpt = dfpt[['date','asset','qty','total','cost','pnl']]

dfpt = dfpt.groupby(['date', 'asset']).agg({'qty': 'sum', 'total': 'sum', 'cost': 'sum', 'pnl': 'sum'})


dfpt.rename(columns={'date': 'Disposal Date', 'asset': 'Asset', 'qty': 'Quantity', 'total': 'Value', 'cost': 'Pooled Cost', 'pnl': 'Gains/Losses'}, inplace=True)
dfpt.to_csv('pnl-rpt-agg.csv')


In [500]:
dfpt

Quantity          Value    Pooled Cost  Gains/Losses
date       asset                                                          
2020-11-13 BTC        0.401707    4965.834791    4190.006644    775.828147
2020-11-14 BTC        0.788639    9614.914965    8474.958627   1139.956338
2020-11-15 BTC        1.216196   14814.242376   13533.118848   1281.123528
2020-11-16 BTC        1.274451   15670.672550   14608.092953   1062.579597
2020-11-17 BTC        0.532474    6729.287915    6266.713746    462.574169
...                        ...            ...            ...           ...
2021-02-16 BTC        5.823608  205575.019732  200575.487241   4999.532491
           DOT     8656.800000  183369.971311  173784.460788   9585.510523
2021-02-18 ALGO   73815.000000   76126.178660   72614.895587   3511.283073
           BTC        5.901960  220174.971833  219992.723679    182.248153
           GRT    89544.000000  144480.330618  140562.873169   3917.457450

[136 rows x 4 columns]

In [464]:
dfr.pnl.sum()

397598.58947253367

In [466]:
dfr.total.sum()

20328784.41229661

In [468]:
dfr.cost.sum()

19931185.82282408

In [471]:
dfr.cost.sum()

11631597.846199984

In [472]:
dfr.cost.sum()

8299587.976624095

In [325]:
dfr


836262.4205467204